In [4]:
from tensorflow.keras.models import load_model
import joblib
import cv2
import numpy as np
model = load_model('emotion_recognition_model.h5')
le = joblib.load('label_encoder.pkl')



In [9]:

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Draw bounding box
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Preprocess the face for the model
        face_roi = gray_frame[y:y+h, x:x+w]
        resized_face = cv2.resize(face_roi, (48, 48))
        normalized_face = resized_face / 255.0
        input_face = normalized_face.reshape(1, 48, 48, 1)

        # Predict emotion
        predictions = model.predict(input_face)
        predicted_class = predictions.argmax()
        predicted_emotion = le.inverse_transform([predicted_class])[0]

        # Add emotion text above the bounding box
        cv2.putText(frame, predicted_emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Emotion Recognition', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 72ms/step
